In [ ]:
import yfinance as yf
import pandas as pd 
import numpy as np  
from sklearn.metrics import mean_squared_error  
from sklearn.preprocessing import MinMaxScaler 
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout  
from tensorflow.keras.callbacks import EarlyStopping #prevent overfitting (when model stops improving stop training)
import plotly.graph_objs as go 
from tensorflow.keras import Input 
from datetime import datetime, timedelta




In [ ]:
print("Enter Ticker Name for Stock: ") 
x = input() 
print("Enter Start Date (YYYY-MM-DD): ")  
startDate = input() 
print("Enter End Date (YYYY-MM-DD): ")  
endDate = input()
stock_data = yf.download(x, start = startDate, end = endDate) 

In [ ]:
close_prices = stock_data['Close'].values #price prediction data, gets closing data

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1)) ## Initializes a scaler to scale values between 0 and 1 with a standard deviation of 1
scaled_data = scaler.fit_transform(close_prices.reshape(-1, 1)) 

In [ ]:
train_size = int(len(scaled_data) * 0.8) #calculates number of data points used in training set (80%) 
train_data, test_data = scaled_data[:train_size], scaled_data[train_size:]  

In [ ]:
def create_sequences(data, seq_length): 
    x,y = [],[] 
    for i in range (seq_length, len(data)):  
        x.append(data[i-seq_length:i, 0]) 
        y.append(data[i, 0]) 
    return np.array(x), np.array(y) 

In [ ]:
seq_length = 100  
x_train, y_train = create_sequences(train_data, seq_length)
x_test, y_test = create_sequences(test_data, seq_length)


In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
model = Sequential([
    Input(shape=(x_train.shape[1], 1)),  # Input shape for LSTM
    LSTM(units=100, return_sequences=True),  # First LSTM layer
    Dropout(0.2),  # Dropout for regularization
    LSTM(units=100, return_sequences=False),  # Second LSTM layer
    Dropout(0.2),  # Dropout for regularization
    Dense(units=1)  # Output layer
])

model.save('stock_predictor_model.keras')

model.compile(optimizer='adam', loss='mean_squared_error')

Early Stoppage 

In [ ]:
# Define Early Stopping
early_stopping = EarlyStopping(
    monitor='val_loss',         
    patience=5,                 
    restore_best_weights=True   
) 


In [ ]:
history = model.fit(
    x_train, y_train,
    validation_split=0.2,  
    epochs=100, #epoch = 1 complete pass through training dataset            
    batch_size=32,        
    callbacks=[early_stopping]
)

In [ ]:
predictions = model.predict(x_test)

predictions = scaler.inverse_transform(predictions)

y_test_scaled = scaler.inverse_transform(y_test.reshape(-1, 1))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=stock_data.index[-len(y_test):], y=y_test_scaled.flatten(), mode='lines', name='Actual Price'))
fig.add_trace(go.Scatter(x=stock_data.index[-len(y_test):], y=predictions.flatten(), mode='lines', name='Predicted Price'))
fig.update_layout(title=x + ' Stock Price Prediction', xaxis_title='Date', yaxis_title='Stock Price (USD)')
fig.show()

GET MSE and RMSE

In [ ]:
# Calculate MSE and RMSE
mse = mean_squared_error(y_test_scaled, predictions)
rmse = np.sqrt(mse)

print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')

Calculate range to determine RMSE performance

In [ ]:
min_value_range = np.min(close_prices)
max_value_range = np.max(close_prices)
range_value = max_value_range - min_value_range 

mean_close_price = np.mean(close_prices)
rmse_percentage = (rmse / mean_close_price) * 100
print(f"RMSE as % of Mean: {rmse_percentage}%")


Get next day future stock price based on previous 100 days of trading info 

In [ ]:
new_end_date = datetime.today()
new_start_date = new_end_date - timedelta(days=100) 

new_end_date_str = new_end_date.strftime('%Y-%m-%d')
new_start_date_str = new_start_date.strftime('%Y-%m-%d')

stock_data_new = yf.download(x, start = new_start_date , end = new_end_date) 


In [ ]:
close_prices_new = stock_data_new['Close'].values.reshape(-1,1) 

scaled_data_new = scaler.fit_transform(close_prices_new)

In [ ]:
new_model = load_model('stock_predictor_model.keras')  


In [ ]:
last_100_days_new = scaled_data_new[-100:]  
last_100_days_new = np.reshape(last_100_days_new, (1, last_100_days_new.shape[0], 1))


In [ ]:
predicted_price_new = model.predict(last_100_days_new)

predicted_price_new_actual = scaler.inverse_transform(predicted_price_new)  

y_test_scaled_new = scaler.inverse_transform(y_test.reshape(-1, 1))


print(f"Predicted next day's price: {predicted_price_new_actual[0][0]:.2f}")
